In [4]:
import pandas as pd

# ---- Bước 1: Config ----
fps = 30  # Số khung hình trên giây của video

# ---- Bước 2: Hàm chuyển đổi time → frame ----
def time_to_frame(time_str: str, fps: int) -> int:
    """
    Chuyển thời gian dạng "HH:MM:SS,mmm" thành chỉ số frame (zero‑based).
    Ví dụ: "00:41:26,000" tại 30 FPS → (41*60 + 26)*30 = 74580.
    """
    h, m, s_ms = time_str.split(':')
    s, ms = s_ms.split(',')
    total_seconds = int(h) * 3600 + int(m) * 60 + int(s) + int(ms) / 1000.0
    return int(total_seconds * fps)


# ---- Bước 3: Đọc file CSV ----
# Keypoint CSV phải có cột 'frame_id' (giá trị frame gốc)
df_keypoint = pd.read_csv("video_1.csv")

# Timeline CSV
df_timeline = pd.read_csv(r"C:\Users\Admin\Downloads\ISAS\Train Data\timetable\csv\1.csv", encoding="utf-8-sig")
df_timeline.rename(columns=lambda x: x.strip(), inplace=True)  # strip BOM/whitespace


# ---- Bước 4: Cắt theo giá trị frame_id ----
cut_segments = []

for idx, row in df_timeline.iterrows():
    start_time = row["Start Time"]
    end_time   = row["End Time"]
    label      = row["Text"]

    # Chuyển time → chỉ số frame
    start_f = time_to_frame(start_time, fps)
    end_f   = time_to_frame(end_time, fps)

    # Lọc tất cả bản ghi có frame_id trong [start_f, end_f]
    mask = (df_keypoint["frame_id"] > start_f) & (df_keypoint["frame_id"] <= end_f)
    segment = df_keypoint.loc[mask].copy()

    if segment.empty:
        print(f"⚠️ Timeline {idx}: Không có frame_id nào trong khoảng {start_f}→{end_f} ({label})")
        continue

    # Gắn cột Timeline_Label
    segment["Action Label"] = label
    cut_segments.append(segment)

    print(f"✅ Cắt timeline {idx}: {label} | frame_id {start_f} → {end_f} ({len(segment)} dòng)")


# ---- Bước 5: Gộp và lưu kết quả ----
if cut_segments:
    df_cut = pd.concat(cut_segments, ignore_index=True)
    df_cut.to_csv("keypoints_cut_by_frame_id_1.csv", index=False)
    print(f"\n✅ Đã lưu tổng cộng {len(df_cut)} dòng vào file: keypoints_cut_by_frame_id.csv")
else:
    print("❌ Không có đoạn nào được cắt! Vui lòng kiểm tra lại timeline và frame_id.")


✅ Cắt timeline 0: Walking | frame_id 0 → 7350 (7350 dòng)
✅ Cắt timeline 1: Head banging | frame_id 7440 → 9420 (1980 dòng)
✅ Cắt timeline 2: Sitting quietly | frame_id 9540 → 16650 (7110 dòng)
✅ Cắt timeline 3: Biting | frame_id 16740 → 19410 (2670 dòng)
✅ Cắt timeline 4: Using phone | frame_id 19860 → 25410 (5550 dòng)
✅ Cắt timeline 5: Throwing things | frame_id 25560 → 27420 (1860 dòng)
✅ Cắt timeline 6: Eating snacks | frame_id 27930 → 35400 (7470 dòng)
✅ Cắt timeline 7: Attacking | frame_id 35970 → 38400 (2430 dòng)
✅ Cắt timeline 8: Walking | frame_id 38550 → 45750 (7200 dòng)
✅ Cắt timeline 9: Head banging | frame_id 46050 → 47820 (1770 dòng)
✅ Cắt timeline 10: Sitting quietly | frame_id 48030 → 55170 (7140 dòng)
✅ Cắt timeline 11: Biting | frame_id 55260 → 58140 (2880 dòng)
✅ Cắt timeline 12: Using phone | frame_id 58380 → 63750 (5370 dòng)
✅ Cắt timeline 13: Throwing things | frame_id 63960 → 66570 (2610 dòng)
✅ Cắt timeline 14: Eating snacks | frame_id 66960 → 74160 (7200 dò

In [5]:
import pandas as pd
import numpy as np

# Đọc dữ liệu
df_cut['Action Label'] = df_cut['Action Label'].fillna('Unknown').astype(str)
y_all = df_cut['Action Label'].values

# Loại bỏ cột frame_id khỏi features
feature_cols = [col for col in df_cut.columns if col not in ['frame_id', 'Action Label']]
X_all = df_cut[feature_cols].values

sequence_length = 30  # 30 frame = 1s

X_seq_avg = []
y_seq_majority = []
seconds_list = []

for idx, i in enumerate(range(0, len(X_all) - sequence_length + 1, sequence_length)):
    # Trung bình từng cột feature
    avg_features = np.mean(X_all[i:i+sequence_length], axis=0)
    X_seq_avg.append(avg_features)
    
    # Majority voting nhãn
    labels_in_seq = y_all[i:i+sequence_length]
    unique_labels, counts = np.unique(labels_in_seq, return_counts=True)
    majority_label = unique_labels[np.argmax(counts)]
    y_seq_majority.append(majority_label)
    
    # Seconds = chỉ số đoạn (giây)
    #seconds_list.append(idx)

# Chuyển sang DataFrame
df_new = pd.DataFrame(X_seq_avg, columns=feature_cols)
#df_new['Seconds'] = seconds_list
df_new['Action Label'] = y_seq_majority

print(df_new.shape)
print(df_new.head())
df_new.to_csv('per_second_cut_1.csv', index=False)


(2419, 35)
       nose_x      nose_y  left_eye_x  left_eye_y  right_eye_x  right_eye_y  \
0  516.450000  265.083333  520.100000  257.191667   518.083333   257.004167   
1  326.020833  277.883333  329.508333  271.516667   328.570833   271.183333   
2  153.764583  300.666667  161.093750  292.316667   154.585417   292.366667   
3   78.643750  314.616667   91.550000  302.525000    64.617708   304.666667   
4  254.229167  298.783333  257.966667  285.566667   235.545833   289.266667   

   left_ear_x  left_ear_y  right_ear_x  right_ear_y  ...  right_hip_y  \
0  540.008333  262.270833   563.083333   258.266667  ...   440.225000   
1  346.266667  280.108333   373.600000   274.783333  ...   441.558333   
2  190.558333  297.900000   187.395833   294.441667  ...   463.325000   
3  114.677083  306.675000    46.083333   312.225000  ...   525.100000   
4  263.804167  290.816667   194.918750   303.925000  ...   552.550000   

   left_knee_x  left_knee_y  right_knee_x  right_knee_y  left_ankle_x  \
0 

In [8]:
import pandas as pd

def check_frame_continuity(df_cut, frame_col='frame', export_csv=False, output_file='frame_gap_errors.csv'):
    """
    Kiểm tra cột frame có liền mạch không (mỗi frame tăng đều 1).
    
    Với mỗi đoạn nhảy frame, in ra cả 2 dòng (frame trước và sau).
    
    Tham số:
    - df_cut: DataFrame gốc
    - frame_col: Tên cột chứa chỉ số frame (mặc định là 'frame')
    - export_csv: Nếu True, xuất các dòng lỗi ra CSV
    - output_file: Tên file CSV để lưu
    """
    # Tính hiệu giữa các frame liên tiếp
    frame_diff = df_cut[frame_col].diff()

    # Tìm index các dòng không liền mạch (diff khác 1), bỏ dòng đầu vì diff NaN
    gap_indices = df_cut.index[frame_diff != 1].tolist()
    gap_indices = [idx for idx in gap_indices if idx != 0]

    if not gap_indices:
        print("✅ Frame liền mạch. Không có lỗi.")
        return []

    print(f"❌ Phát hiện {len(gap_indices)} đoạn frame không liền mạch:\n")

    # In ra 2 dòng của từng đoạn ngách quảng
    for idx in gap_indices:
        prev_idx = idx - 1
        print(f"Đoạn lỗi tại index {prev_idx} và {idx}:")
        print(df_cut.iloc[[prev_idx, idx]])
        print("-" * 60)

    if export_csv:
        # Lấy cả 2 dòng (trước và sau) của từng lỗi
        rows_to_export = df_cut.iloc[[idx - 1 for idx in gap_indices] + gap_indices]
        rows_to_export.to_csv(output_file, index=False)
        print(f"\n✅ Đã lưu các đoạn lỗi vào file: {output_file}")

    return gap_indices

# Cách sử dụng:

# Đọc file CSV
df_cut = pd.read_csv('keypoints_cut_by_frame_id_3.csv')

# Gọi hàm kiểm tra
check_frame_continuity(df_cut, frame_col='frame_id', export_csv=True, output_file='frame_gap_errors.csv')


❌ Phát hiện 16 đoạn frame không liền mạch:

Đoạn lỗi tại index 18239 và 18240:
       frame_id  nose_x  nose_y  left_eye_x  left_eye_y  right_eye_x  \
18239     18240   538.0  435.50       546.0     430.000        540.5   
18240     18781   911.0  142.25       911.5     134.375        916.0   

       right_eye_y  left_ear_x  left_ear_y  right_ear_x  ...  right_hip_y  \
18239        427.0       575.0      440.50        567.0  ...       598.00   
18240        134.0       917.5      140.75        970.0  ...       392.25   

       left_knee_x  left_knee_y  right_knee_x  right_knee_y  left_ankle_x  \
18239        507.0        641.0        506.25         581.0         479.5   
18240        902.5        496.0        976.00         516.0         888.0   

       left_ankle_y  right_ankle_x  right_ankle_y  Action Label  
18239         624.0          497.5          622.0  Head banging  
18240         596.5          977.0          653.5       Walking  

[2 rows x 36 columns]
-------------------

[18240,
 37440,
 38370,
 48270,
 49950,
 62970,
 67560,
 69540,
 71280,
 81210,
 87030,
 87780,
 88860,
 89940,
 92940,
 96600]